In [1]:
import pandas as pd
from pathlib import Path


EXTRACT DATA TO CSV

In [ ]:


excel_dir = Path(r"C:\Users\do.hai-minh\Downloads\data_test")
output_dir = Path(r"C:\Users\do.hai-minh\Downloads\data_test\output_file")
output_dir.mkdir(parents=True, exist_ok=True)

# Duyệt từng file Excel
for excel_file in excel_dir.glob("*.xlsx"):
    print(f"Processing: {excel_file.name}")

    # Lấy danh sách sheet
    xls = pd.ExcelFile(excel_file)

    for idx, sheet_name in enumerate(xls.sheet_names):
        print(f"  - Exporting sheet: {sheet_name}")

        # Sheet đầu tiên: skip 2 dòng
        if idx == 0:
            df = pd.read_excel(
                excel_file,
                sheet_name=sheet_name,
                skiprows=2
            )
        else:
            df = pd.read_excel(
                excel_file,
                sheet_name=sheet_name
            )

        # Tên file CSV (loại bỏ ký tự đặc biệt trong tên sheet)
        safe_sheet_name = sheet_name.replace(" ", "_")
        csv_file = output_dir / f"{excel_file.stem}_{safe_sheet_name}.csv"

        df.to_csv(csv_file, index=False, encoding="utf-8-sig")

print("Done.")


Processing: 2007 Contoso Data.xlsx
  - Exporting sheet: FACT Sales
  - Exporting sheet: FACT Sales Quota
Processing: 2008 Contoso Data.xlsx
  - Exporting sheet: FACT Sales
  - Exporting sheet: FACT Sales Quota
Processing: 2009 Contoso Data.xlsx
  - Exporting sheet: FACT Sales
  - Exporting sheet: FACT Sales Quota
Processing: Contoso Lookup Tables.xlsx
  - Exporting sheet: DIM Date
  - Exporting sheet: DIM Product
  - Exporting sheet: DIM Product Sub Category
  - Exporting sheet: DIM Product Category
  - Exporting sheet: DIM Geography
  - Exporting sheet: DIM Channel
  - Exporting sheet: DIM Store
Processing: Store_full.xlsx
  - Exporting sheet: Sheet1
Done.


In [ ]:

csv_dir = Path(r"C:\Users\do.hai-minh\Downloads\data_test\output_file")
output_file = Path(r"C:\Users\do.hai-minh\Downloads\data_test\final_union.csv")

dfs = []

for csv_file in csv_dir.glob("*Contoso Data_FACT_Sales.csv"):
    print(f"Reading: {csv_file.name}")
    df = pd.read_csv(csv_file)
    dfs.append(df)

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print("Done.")


NameError: name 'Path' is not defined

DIM PRODUCT

In [10]:

df_product = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Contoso Lookup Tables_DIM_Product.csv')
df_product_category = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Contoso Lookup Tables_DIM_Product_Category.csv')
df_product_sub_category = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Contoso Lookup Tables_DIM_Product_Sub_Category.csv')
df_date = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Contoso Lookup Tables_DIM_Date.csv')


In [43]:
df_product_full = df_product_category.merge(df_product_sub_category, how='left', left_on='ProductCategoryKey', right_on='ProductCategoryKey')
df_product_full = df_product.merge(df_product_full, how='left', left_on='ProductSubcategoryKey', right_on='ProductSubcategoryKey')

weight_unit_measure = {
    'grams' : 1
    , 'ounces' : 28.3495
    , 'pounds' : 453.592
}

df_product_full['WeightGram'] = df_product_full['Weight'] * df_product_full['WeightUnitMeasureID'].map(weight_unit_measure)


# df_product_full[df_product_full['WeightUnitMeasureID'].isnull()]
display(df_product_full.head(3))

df_product_full.to_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Dim_product.csv', index=False, encoding='utf-8-sig')  

,ProductKey,ProductName,ProductDescription,ProductSubcategoryKey,Manufacturer,BrandName,ClassName,ColorName,Size,SizeRange,Weight,WeightUnitMeasureID,UnitCost,UnitPrice,ProductCategoryKey,ProductCategoryName,ProductSubcategoryName,WeightGram
0,4,Contoso 2G MP3 Player E200 Silver,"2GB flash memory, LCD display, plays MP3 and WMA",1,"Contoso, Ltd",Contoso,Economy,Silver,2.2 x 2.2 x 4,,4.5,ounces,11.00,21.57,1,Audio,MP4&MP3,127.57275
1,8,Contoso 4G MP3 Player E400 Silver,"4GB flash memory and FM Radio, LCD Display wit...",1,"Contoso, Ltd",Contoso,Economy,Silver,0.8 x 3.6 x 1.1,,5.6,ounces,30.58,59.99,1,Audio,MP4&MP3,158.75720
2,12,Contoso 4GB Flash MP3 Player E401 Blue,"1.8'' color LCD, play MP3, WMA and Video MTV, ...",1,"Contoso, Ltd",Contoso,Economy,Blue,0.8 x 3.6 x 1.1,,7.4,ounces,35.72,77.68,1,Audio,MP4&MP3,209.78630


DIM DATE

In [ ]:
df_date['DateKey'] = (pd.to_datetime(df_date['Date']) - pd.Timestamp('1899-12-30')).dt.days

display(df_date.head(3))


,Date,CalendarYear,CalendarHalfYear,CalendarQuarter,CalendarMonth,CalendarWeek,CalendarDayofWeek,FiscalYear,FiscalHalfYear,FiscalQuarter,FiscalMonth,IsWorkday,EuropeSeason,NorthAmericaSeason,AsiaSeason,DateKey
0,1/1/2007,2007,H1,Q1,January,Week 1,Monday,2007,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday,39083
1,1/2/2007,2007,H1,Q1,January,Week 1,Tuesday,2007,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday,39084
2,1/3/2007,2007,H1,Q1,January,Week 1,Wednesday,2007,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday,39085


In [41]:
df_fact_sale = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\2007 Contoso Data_FACT_Sales.csv')
df_fact_sale.merge(df_date, how='left', left_on='DateKey', right_on='DateKey').head(3)


,SalesKey,DateKey,channelKey,StoreKey,ProductKey,PromotionKey,UnitCost,UnitPrice,SalesQuantity,ReturnQuantity,...,CalendarWeek,CalendarDayofWeek,FiscalYear,FiscalHalfYear,FiscalQuarter,FiscalMonth,IsWorkday,EuropeSeason,NorthAmericaSeason,AsiaSeason
0,3643,39814,4,310,1532,14,128.76,280.00,26,0,...,Week 1,Thursday,2009,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday
1,7425,39814,4,310,2517,14,1.71,3.35,1040,0,...,Week 1,Thursday,2009,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday
2,14378,39814,1,27,72,20,22.05,47.95,18,0,...,Week 1,Thursday,2009,H1,Q1,Month 1,WorkDay,Holiday,Spring/Back to Business,Holiday


In [29]:
df_date.to_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Dim_date.csv', index=False, encoding='utf-8-sig')    

DIM STORE

In [56]:
df_store = pd.read_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Store_full.csv')

df_store[['Longitude', 'Latitude']] = df_store['GeoLocation'].str.extract(r'\(([^ ]+) ([^)]+)\)')
df_store[['Longitude', 'Latitude']] = df_store[['Longitude', 'Latitude']].astype(float)

display(df_store.head(3))
print(df_store.dtypes)

df_store.to_csv(r'C:\Users\do.hai-minh\Downloads\data_test\output_file\Dim_store_full.csv', index=False, encoding='utf-8-sig')

,StoreKey,GeographyKey,StoreManager,StoreType,StoreName,StoreDescription,Status,OpenDate,CloseDate,EntityKey,...,EmployeeCount,SellingAreaSize,LastRemodelDate,GeoLocation,Geometry,ETLLoadID,LoadDate,UpdateDate,Longitude,Latitude
0,15,678,48,Store,Contoso Renton Store,Contoso Renton Store,On,2/24/2004 0:00,NaN,649,...,22.0,500,6/17/2009 0:00,POINT (-122.189 47.4894),POINT (-122.189 47.4894),1,7/10/2009 0:00,6/17/2009 0:00,-122.189,47.4894
1,17,551,135,Store,Contoso Spokane Store,Contoso Spokane Store,On,3/26/2004 0:00,NaN,651,...,25.0,700,6/17/2009 0:00,POINT (-117.414 47.6566),POINT (-117.414 47.6566),1,7/10/2009 0:00,6/17/2009 0:00,-117.414,47.6566
2,20,824,52,Store,Contoso Englewood Store,Contoso Englewood Store,On,9/29/2004 0:00,NaN,654,...,33.0,560,6/17/2009 0:00,POINT (-104.961 39.6541),POINT (-104.961 39.6541),1,7/10/2009 0:00,6/17/2009 0:00,-104.961,39.6541


StoreKey              int64
GeographyKey          int64
StoreManager          int64
StoreType            object
StoreName            object
StoreDescription     object
Status               object
OpenDate             object
CloseDate            object
EntityKey             int64
ZipCode             float64
ZipCodeExtension    float64
StorePhone           object
StoreFax             object
AddressLine1         object
AddressLine2         object
CloseReason          object
EmployeeCount       float64
SellingAreaSize       int64
LastRemodelDate      object
GeoLocation          object
Geometry             object
ETLLoadID             int64
LoadDate             object
UpdateDate           object
Longitude           float64
Latitude            float64
dtype: object
